In [7]:
import cv2
import os
from scipy.spatial import distance as dist
from imutils import face_utils # for face detection
import imutils
import dlib
import datetime
import playsound
os.chdir('./data/')

In [8]:
ALARM_ON = False
MODEL_PATH = '../../../../dlib/shape_predictor_68_face_landmarks.dat'
ALARM_PATH = 'assets_alarm.mp3'

In [9]:
'''
calculate the distance between eyes landmarks

the landmarks are:
  1 2
0     3
  5 4
'''

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

In [19]:
def detect():
    THRESH = 0.25 # threshold for the eye aspect ratio
    FRAME_CHECK = 30 # after 30 frames without a blink, we will say that the person is drowsy
    Detect = dlib.get_frontal_face_detector()
    Predict = dlib.shape_predictor(MODEL_PATH)

    (LStart, LEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"] # get landmarks for the left eye
    (RStart, REnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"] # get landmarks for the right eye

    cap = cv2.VideoCapture(0)
    flag = 0
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        
        frame = imutils.resize(frame, width=650, height=650)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = Detect(gray, 0)

        for face in faces:
            landmarks = Predict(gray, face)
            landmarks = face_utils.shape_to_np(landmarks) # convert from list to numpy array

            left_eye = landmarks[LStart:LEnd]
            right_eye = landmarks[RStart:REnd]

            left_EAR = eye_aspect_ratio(left_eye)
            right_EAR = eye_aspect_ratio(right_eye)
            EAR = (left_EAR + right_EAR) / 2.0  # Take the average of the two eyes

            left_eye_hull = cv2.convexHull(left_eye)
            right_eye_hull = cv2.convexHull(right_eye)

            cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)

            if EAR < THRESH:
                flag += 1
                if flag >= FRAME_CHECK:
                    if ALARM_ON == False:
                        ALARM_ON = True
                    
                    cv2.putText(frame, "Drowsy!!!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    playsound.playsound(ALARM_PATH,ALARM_ON)
            
            else:
                flag = 0
                ALARM_ON = False
                cv2.putText(frame, "Safe", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
                break
        

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    detect()

[ WARN:0@1736.662] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
